In [2]:
import pandas as pd

# Load WRDS extract
df = pd.read_excel("01_wrds-original.xlsx")

df["revt"] = df["revt"].round(0)
df["at"] = df["at"].round(0)
df["lt"] = df["lt"].round(0)
df["act"] = df["act"].round(0)
df["lct"] = df["lct"].round(0)
df["revt"] = df["revt"].round(0)
df["cogs"] = df["cogs"].round(0)
df["xsga"] = df["xsga"].round(0)
df["oiadp"] = df["oiadp"].round(0)
df["ni"] = df["ni"].round(0)

# --- Derived metrics ---
df["gross_margin"] = ((df["revt"] - df["cogs"]) / df["revt"]).map(lambda x: f"{x:.2%}")
df["operating_margin"] = (df["oiadp"] / df["revt"]).map(lambda x: f"{x:.2%}")
df["net_margin"] = (df["ni"] / df["revt"]).map(lambda x: f"{x:.2%}")
df["eq"] = (df["at"] - df["lt"]).round(0)
df["return_on_equity"] = (df["ni"] / df["eq"]).map(lambda x: f"{x:.2%}")
df["current_ratio"] = (df["act"] / df["lct"]).round(2)
df["debt_to_equity_ratio"] = (df["lt"] / df["eq"]).round(2)
df["asset_turnover"] = (df["revt"] / df["at"]).round(2)

# --- Keep only needed variables ---
vars_needed = [
    "Year","tic","revt","gross_margin","operating_margin","net_margin",
    "return_on_equity","current_ratio","debt_to_equity_ratio","asset_turnover"]
df["Year"] = pd.to_datetime(df["date"]).dt.year
df_clean = df[vars_needed]

# --- Pivot to Year × Company × Metrics ---
pivot = df_clean.pivot(index="Year", columns="tic")
pivot.columns = [f"{var}_{tic}" for var, tic in pivot.columns]

# --- Save with proper column widths ---
with pd.ExcelWriter("01_financial-ratios-clean.xlsx", engine="xlsxwriter") as writer:
    pivot.reset_index().to_excel(writer, sheet_name="Ratios", index=False)
    worksheet = writer.sheets["Ratios"]
    for i, col in enumerate(pivot.reset_index().columns):
        max_len = max(pivot.reset_index()[col].astype(str).map(len).max(), len(col)) + 2
        worksheet.set_column(i, i, max_len)

print("Completed")

Completed
